Import das bibliotecas necessárias


In [83]:
import psycopg2 as pg
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

Configuração da conexão com o banco de dados PostgreSQL

In [84]:
try:
  conn = pg.connect(database="projeto-fbd",host="localhost",port=5432,user='postgres',password='198019')
  print('conexão concluida')
except(Exception) as error:
  #print(error)
  print('conexão não concluida')

conexão concluida


Criação de widgets

In [85]:
try:
    meta_calorica = widgets.Text(disabled=False, placeholder='Meta calórica', value='', description='Meta Calórica')
    quantidade_refeicoes = widgets.Text(disabled=False, description='Quantidade', value='', placeholder='Quantidade de Refeições')
    intervalo_refeicoes = widgets.Text(disabled=False, description='Intervalo', value='',placeholder='Intervalo entre Refeições')
    objetivo = widgets.Text(disabled=False, placeholder='Objetivo da Dieta', value='', description='Objetivo')
    dieta_id = widgets.Text(disabled=False, description='ID', value='', placeholder='ID da Dieta')
    cpf_usuario = widgets.Text(disabled=False, placeholder='CPF: ', value='', description='CPF')
    alimentos = widgets.Textarea(disabled=False, value='', placeholder='Alimentos: ', description='Alimentos')
except Exception as error:
    print('Falha:', error)


Criação dos botões e declaração das suas funções


In [86]:
# Criação dos botões
createButton = widgets.Button(description="Cadastrar")
readButton = widgets.Button(description="Consultar", value='Preencha os campos acima para consultar')
updateButton = widgets.Button(description="Atualizar", value='Preencha os campos acima para atualizar')
deleteButton = widgets.Button(description="Deletar", value='Preencha os campos acima para deletar')
readButtonPerID = widgets.Button(description="Consultar por ID", value='Preencha o campo ID acima para consultar')
output = widgets.Output()

def onCreateButton(b):
    try:
       
        meta_calorica_value = float(meta_calorica.value)
        quantidade_refeicoes_value = int(quantidade_refeicoes.value)
        intervalo_refeicoes_value = int(intervalo_refeicoes.value)
        objetivo_value = str(objetivo.value)
        dieta_id_value = int(dieta_id.value)
        cpf_usuario_value = str(cpf_usuario.value)
        alimentos_value = str(alimentos.value)

        cursor = conn.cursor()
        cursor.execute('INSERT INTO Dieta (Meta_Calorica, Quantidade_de_refeicoes, Intervalo_entre_refeicoes, Objetivo, DietaId, CpfUsuario) VALUES (%s, %s, %s, %s, %s, %s)',
                       (meta_calorica_value, quantidade_refeicoes_value, intervalo_refeicoes_value, objetivo_value, dieta_id_value, cpf_usuario_value))
        
        cursor.execute('INSERT INTO Dieta_Alimentos (Alimentos, DietaID) VALUES (%s, %s)', (alimentos_value, dieta_id_value))
        conn.commit()
        
        print('Dados inseridos com sucesso!')

        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)

        with output:
            output.clear_output()
            display(df)
    except (Exception, pg.Error) as error:
        conn.rollback()
        with output:
            output.clear_output()
            print('Falha ao inserir dados', error)
    finally:
        cursor.close()


def onReadButton(b):
    try:
        cursor = conn.cursor()
        cursor.execute('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId')
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)
        with output:
            output.clear_output(wait=True)
            display(df)
    except (Exception, pg.Error) as error:
        conn.rollback()
        with output:
            output.clear_output()
            print('Falha ao ler dados', error)
    finally:
        cursor.close()

def onUpdateButton(b):
    try:

        meta_calorica_value = float(meta_calorica.value)
        quantidade_refeicoes_value = int(quantidade_refeicoes.value)
        intervalo_refeicoes_value = int(intervalo_refeicoes.value)
        objetivo_value = str(objetivo.value)
        dieta_id_value = int(dieta_id.value)
        cpf_usuario_value = str(cpf_usuario.value)
        alimentos_value = str(alimentos.value)

        cursor = conn.cursor()
        cursor.execute('UPDATE Dieta SET Meta_Calorica = %s, Quantidade_de_refeicoes = %s, Intervalo_entre_refeicoes = %s, Objetivo = %s, CpfUsuario = %s WHERE DietaId = %s',
                       (meta_calorica_value, quantidade_refeicoes_value, intervalo_refeicoes_value, objetivo_value, cpf_usuario_value, dieta_id_value))
        
        cursor.execute('UPDATE Dieta_Alimentos SET Alimentos = %s WHERE DietaID = %s', (alimentos_value, dieta_id_value))
        
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)
        
        with output:
            output.clear_output()
            display(df)
    
    except (Exception, pg.Error) as error:
        conn.rollback()
        with output:
            output.clear_output()
            print('Falha ao atualizar dados', error)
    finally:
        cursor.close()

def onDeleteButton(b):
    try:
        cursor = conn.cursor()
        cursor.execute('DELETE FROM Dieta_Alimentos WHERE DietaId = %s', (dieta_id,))
        cursor.execute('DELETE FROM Dieta WHERE DietaId = %s', (dieta_id,))
        
        df = pd.read_sql_query('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId', conn)
        
        with output:
            output.clear_output()
            display(df)
    
    except (Exception, pg.Error) as error:
        conn.rollback()
        with output:
            output.clear_output()
            print('Falha ao deletar dados', error)
    finally:
        cursor.close()


def onReadButtonPerID(b):
    try:
        dieta_id_value = int(dieta_id.value)
        cursor = conn.cursor()
        cursor.execute('SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId where d.DietaId = %s', (dieta_id_value,))
        sql_code = 'SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId where d.DietaId = %s' % (dieta_id_value)
        print(sql_code)
        df = pd.read_sql_query(sql_code, conn)
        with output:
            output.clear_output(wait=True)
            display(df)
    except (Exception, pg.Error) as error:
        conn.rollback()
        with output:
            output.clear_output()
            print('Falha ao ler dados', error)
    finally:
        cursor.close()

createButton.on_click(onCreateButton)
readButton.on_click(onReadButton)
updateButton.on_click(onUpdateButton)
deleteButton.on_click(onDeleteButton)
readButtonPerID.on_click(onReadButtonPerID)

# Display buttons
#display(createButton, readButton, updateButton, deleteButton, output)


SELECT d.*, a.alimentos FROM Dieta d JOIN Dieta_Alimentos a ON a.DietaId = d.DietaId where d.DietaId = 5


/tmp/ipykernel_21448/3783819706.py:118: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_code, conn)


Exibir widgets

In [87]:
with output:
    display(meta_calorica,quantidade_refeicoes,intervalo_refeicoes,objetivo,dieta_id,cpf_usuario,alimentos,createButton,readButton,readButtonPerID,updateButton,deleteButton)
output

Output()